In [5]:
import sqlite3

conn = sqlite3.connect('Flower.db')
cursor = conn.cursor()

In [6]:
cursor.execute('''
        CREATE TABLE Flowers(
            ID INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            Type TEXT NOT NULL,
            Source TEXT NOT NULL,
            PurchasePrice REAL,
            SalePrice REAL,
            StockQuantity INTEGER,
            SoldQuantity INTEGER,
            ExpiryDate DATE,
            Description TEXT,
            EntryDate DATE DEFAUL CURRENT_DATE
        );
''')

In [7]:
flowers = [
    ('Rose','Flower','France',1.2,2.5,100,10,'2024-04-08','A beautiful red rose'),
    ('Tulip','Flower','Netherlands',0.8,2.0,150,25,'2024-04-08','A colorful tulip'),
    ('Lily','Flower','China',1.5,3.0,80,5,'2024-04-08','A elegant white lily'),
    ('Daisy','Flower','USA',0.7,1.8,120,15,'2024-04-08','A cheerful daisy flower'),
    ('Orchid','Flower','Brazil',2.0,4.0,50,2,'2024-04-08','A delicate purple orchid'),
]

In [8]:
for flower in flowers:
    cursor.execute('''
INSERT INTO Flowers (Name,Type,Source,PurchasePrice,SalePrice,StockQuantity,SoldQuantity,ExpiryDate,Description)
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
''',flower)
    
conn.commit()

conn.close()

In [9]:
# Querying the database with Chains:pip install lanchain-experimental
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAI

from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Flower.db")

llm = OpenAI(temperature=0,verbose=True)

query_chain = create_sql_query_chain(llm, db)

response = query_chain.invoke({"question": "How many different types of flowers?"})
print(response)

SELECT COUNT(DISTINCT Type) FROM Flowers


In [10]:
response = query_chain.invoke({"question": "Which flower has the least inventory?"})
print(response)
# response = db_chain.run("What is the average selling price?")
# print(response)

SELECT "Name" FROM "Flowers" ORDER BY "StockQuantity" ASC LIMIT 1


In [11]:

from langchain_community.utilities import SQLDatabase

from langchain_openai import OpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType


db = SQLDatabase.from_uri("sqlite:///Flower.db")

llm = OpenAI(temperature=0,verbose=True)

agent_executor = create_sql_agent(
    llm,
    toolkit=SQLDatabaseToolkit(db=db,llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

questions = [
    "Which flower has the least inventory?",
    "What is the average selling price?"
]

for question in questions:
    response = agent_executor.run(question)
    print(response)

/var/folders/tj/g02zt01x1n35z6_cfnf78t840000gn/T/ipykernel_65057/954638947.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run(question)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
Flowers I should query the schema of the Flowers table to see what columns I can use.
Action: sql_db_schema
Action Input: Flowers
CREATE TABLE "Flowers" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Type" TEXT NOT NULL, 
	"Source" TEXT NOT NULL, 
	"PurchasePrice" REAL, 
	"SalePrice" REAL, 
	"StockQuantity" INTEGER, 
	"SoldQuantity" INTEGER, 
	"ExpiryDate" DATE, 
	"Description" TEXT, 
	"EntryDate" NUMERIC, 
	PRIMARY KEY ("ID")
)

/*
3 rows from Flowers table:
ID	Name	Type	Source	PurchasePrice	SalePrice	StockQuantity	SoldQuantity	ExpiryDate	Description	EntryDate
1	Rose	Flower	France	1.2	2.5	100	10	2024-04-08	A beautiful red rose	None
2	Tulip	Flower	Netherlands	0.8	2.0	150	25	2024-04-08	A colorful tulip	None
3	Lily	Flower	China	1.5	3.0	80	5	2024-04-08	A elegant white lily	None
*/ I should query the Flowers table to get the flower with the least inventory.
Action: sql_db_query
Action Input: SELECT Name FROM